In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import random
import numpy as np
import pandas as pd


from pandas.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from sklearn.ensemble import GradientBoostingRegressor

In [3]:
np.warnings.filterwarnings('ignore')

## Utils

### Load Data

In [4]:
# load_data

desired_directory = '/content/drive/MyDrive/1. Projects/Kaggle-birthweight-prediction/data'
os.chdir(desired_directory)
# print("Current working directory:", os.getcwd())

train = pd.read_csv("train.csv", index_col="id")
test = pd.read_csv("test.csv", index_col="id")


In [5]:
train.head()

,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,...,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN,DBWT
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,18.5,0,12,,10,1434,5,99,...,0,0,108,1,1,N,0,F,24,2800
1,1,1,18.3,2,4,1,12,2156,6,39,...,2,1,100,1,1,N,0,M,18,1900
2,1,1,27.3,0,3,2,12,1241,2,31,...,2,2,135,4,1,Y,2,F,27,2960
3,1,1,24.0,0,7,2,4,1649,2,99,...,0,0,111,3,1,N,0,M,29,3657
4,2,1,23.6,0,6,1,3,752,2,31,...,2,0,121,4,1,Y,2,F,37,3742


In [6]:
train.describe()

,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,FEDUC,...,PREVIS,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESARN,WTGAIN,DBWT
count,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,...,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000,108082.000000
mean,1.330046,1.033891,28.848412,1.557336,10.949048,6.570835,1233.242085,4.055162,39.630447,4.903842,...,13.532670,0.236598,1.290409,0.721665,176.276216,1.796691,1.331582,0.258776,31.566949,3260.113654
std,0.765864,0.310660,12.731588,8.217045,19.837302,3.421184,632.722133,1.855129,22.471849,2.303684,...,14.408799,4.681943,4.100238,5.461285,125.323220,1.166049,0.533019,1.987953,19.113690,589.548274
min,1.000000,1.000000,13.200000,0.000000,1.000000,1.000000,0.000000,1.000000,14.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,75.000000,1.000000,1.000000,0.000000,0.000000,227.000000
25%,1.000000,1.000000,22.300000,0.000000,4.000000,4.000000,801.000000,2.000000,28.000000,3.000000,...,9.000000,0.000000,0.000000,0.000000,130.000000,1.000000,1.000000,0.000000,20.000000,2965.000000
50%,1.000000,1.000000,25.800000,0.000000,7.000000,7.000000,1238.000000,4.000000,33.000000,4.000000,...,12.000000,0.000000,1.000000,0.000000,150.000000,1.000000,1.000000,0.000000,30.000000,3300.000000
75%,1.000000,1.000000,31.200000,0.000000,10.000000,10.000000,1735.000000,6.000000,38.000000,6.000000,...,14.000000,0.000000,2.000000,1.000000,182.000000,3.000000,2.000000,0.000000,40.000000,3629.000000
max,9.000000,9.000000,99.900000,99.000000,99.000000,12.000000,9999.000000,7.000000,99.000000,9.000000,...,99.000000,99.000000,99.000000,99.000000,999.000000,9.000000,4.000000,99.000000,99.000000,6840.000000


In [7]:
train.columns

Index(['ATTEND', 'BFACIL', 'BMI', 'CIG_0', 'DLMP_MM', 'DMAR', 'DOB_MM',
       'DOB_TT', 'DOB_WK', 'FAGECOMB', 'FEDUC', 'ILLB_R', 'ILOP_R', 'ILP_R',
       'LD_INDL', 'MAGER', 'MBSTATE_REC', 'MEDUC', 'M_Ht_In', 'NO_INFEC',
       'NO_MMORB', 'NO_RISKS', 'PAY', 'PAY_REC', 'PRECARE', 'PREVIS',
       'PRIORDEAD', 'PRIORLIVE', 'PRIORTERM', 'PWgt_R', 'RDMETH_REC',
       'RESTATUS', 'RF_CESAR', 'RF_CESARN', 'SEX', 'WTGAIN', 'DBWT'],
      dtype='object')

In [8]:
test.head()

,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
id,,,,,,,,,,,,,,,,,,,,,
108082,1,1,23.6,0,11,,8,1231,4,26,...,0,0,0,125,3,1,N,0,M,53
108083,1,1,20.1,0,7,1,4,851,5,32,...,0,2,2,103,4,2,Y,2,F,24
108084,1,1,34.8,30,3,2,12,36,6,24,...,0,2,1,203,1,1,N,0,F,0
108085,1,1,23.4,0,10,2,7,1452,3,32,...,0,1,0,128,1,1,N,0,F,29
108086,1,1,25.4,0,8,,5,803,4,34,...,0,3,0,130,4,1,Y,1,F,18


In [ ]:
# scatter_matrix(train[train.columns[-10:]])

In [ ]:
# scatter_matrix(test)

### WIS_and_coverage

In [ ]:
def WIS_and_coverage(y_true,lower,upper,alpha):

    assert np.isnan(y_true) == False, "y_true contains NaN value(s)"
    assert np.isinf(y_true) == False, "y_true contains inf values(s)"
    assert np.isnan(lower)  == False, "lower interval value contains NaN value(s)"
    assert np.isinf(lower)  == False, "lower interval value contains inf values(s)"
    assert np.isnan(upper)  == False, "upper interval value contains NaN value(s)"
    assert np.isinf(upper)  == False, "upper interval value contains inf values(s)"
    assert alpha > 0 and alpha <= 1,  f"alpha should be (0,1]. Found: {alpha}"

    # WIS for one single row
    score = np.abs(upper-lower)
    if y_true < np.minimum(upper,lower):
        score += ((2/alpha) * (np.minimum(upper,lower) - y_true))
    if y_true > np.maximum(upper,lower):
        score += ((2/alpha) * (y_true - np.maximum(upper,lower)))
    # coverage for one single row
    coverage  = 1 # assume is within coverage
    if (y_true < np.minimum(upper,lower)) or (y_true > np.maximum(upper,lower)):
        coverage = 0
    return score, coverage


### score

In [ ]:
# vectorize the function
v_WIS_and_coverage = np.vectorize(WIS_and_coverage)

def score(y_true,lower,upper,alpha):
    """
    This is an implementation of the Winkler Interval score (https://otexts.com/fpp3/distaccuracy.html#winkler-score).
    The mean over all of the individual Winkler Interval scores (MWIS) is returned, along with the coverage.

    See:
    [1] Robert L. Winkler "A Decision-Theoretic Approach to Interval Estimation", Journal of the American Statistical Association, vol. 67, pp. 187-191 (1972) (https://doi.org/10.1080/01621459.1972.10481224)
    [2] Tilmann Gneiting and Adrian E Raftery "Strictly Proper Scoring Rules, Prediction, and Estimation", Journal of the American Statistical Association, vol. 102, pp. 359-378 (2007) (https://doi.org/10.1198/016214506000001437) (Section 6.2)

    Version: 1.0.4
    Author:  Carl McBride Ellis
    Date:    2023-12-07
    """

    assert y_true.ndim == 1, "y_true: pandas Series or 1D array expected"
    assert lower.ndim  == 1, "lower: pandas Series or 1D array expected"
    assert upper.ndim  == 1, "upper: pandas Series or 1D array expected"
    assert isinstance(alpha, float) == True, "alpha: float expected"

    WIS_scores, coverage = v_WIS_and_coverage(y_true,lower,upper,alpha)
    MWIS      = np.mean(WIS_scores)
    MWIS      = float(MWIS)
    coverage  = coverage.sum()/coverage.shape[0]
    coverage  = float(coverage)

    return MWIS,coverage

### train_calibration_test_split

In [ ]:
def train_calibration_test_split(X, y, test_pct=0.2, calibration_pct=0.5, random_state=42):
    """
    Split data into:
    - x_train, y_train for training the model
    - x_cali, y_cali for calibrating the model
    - x_test, y_test for testing the un-seen data
    """
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_pct, random_state=random_state)
    x_train, x_cali, y_train, y_cali = train_test_split(x_train, y_train, test_size=calibration_pct, random_state=random_state)

    return x_train, x_cali, x_test, y_train, y_cali, y_test


### Preprocessing

In [ ]:
def preparing_data(data, **kwarg):

    X = data[data.columns[:-1]]
    y = data[data.columns[-1]]

    X = pd.get_dummies(X)

    x_train, x_cali, x_test, y_train, y_cali, y_test = train_calibration_test_split(X, y, **kwarg)

    return x_train, x_cali, x_test, y_train, y_cali, y_test

In [ ]:
x_train, x_cali, x_test, y_train, y_cali, y_test = preparing_data(train)

print(f"x_train's shape: {x_train.shape}")
print(f"x_cali's shape: {x_cali.shape}")
print(f"x_test's shape: {x_test.shape}")
print(f"y_train's shape: {y_train.shape}")
print(f"y_cali's shape: {y_cali.shape}")
print(f"y_test's shape: {y_test.shape}")


x_train's shape: (43232, 41)
x_cali's shape: (43233, 41)
x_test's shape: (21617, 41)
y_train's shape: (43232,)
y_cali's shape: (43233,)
y_test's shape: (21617,)


## Train The Model - Testing Phase

### Constants

In [ ]:
# model config

# desired miscoverage error
alpha = 0.1

# desired quanitile levels
quantiles = [0.05, 0.95]

# used to determine the size of test set
test_ratio = 0.2

# output number
i = 0

### Train lower_model and upper_model

In [ ]:
X = train[train.columns[:-1]]
y = train[train.columns[-1]]

X = pd.get_dummies(X)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Each model has to be separate
lower_model = GradientBoostingRegressor(loss="quantile",
                                        alpha=quantiles[0])
# The mid model will use the default loss
mid_model = GradientBoostingRegressor(loss="ls")
upper_model = GradientBoostingRegressor(loss="quantile",
                                        alpha=quantiles[1])

# fit the model
lower_model.fit(x_train, y_train)
upper_model.fit(x_train, y_train)


GradientBoostingRegressor(alpha=0.95, loss='quantile')

In [ ]:

y_test_lower = lower_model.predict(x_test)
y_test_upper = upper_model.predict(x_test)

In [ ]:
Q = 3

y_test_lower_modified = y_test_lower - Q
y_test_upper_modified = y_test_upper + Q

In [ ]:
score_, coverage = score(y_test, y_test_lower_modified, y_test_upper_modified, alpha=0.1)

In [ ]:
score_

2166.9076815185676

In [ ]:
coverage

0.900818800018504

### train full data

In [ ]:
X = train[train.columns[:-1]]
y = train[train.columns[-1]]

X = pd.get_dummies(X)

In [ ]:
# Each model has to be separate
lower_model = GradientBoostingRegressor(loss="quantile",
                                        alpha=quantiles[0])
# The mid model will use the default loss
mid_model = GradientBoostingRegressor(loss="ls")
upper_model = GradientBoostingRegressor(loss="quantile",
                                        alpha=quantiles[1])

# fit the model
lower_model.fit(X, y)
upper_model.fit(X, y)

GradientBoostingRegressor(alpha=0.95, loss='quantile')

In [ ]:
# x_cali y_cali

y_cali_lower_quantile = lower_model.predict(x_cali)
y_cali_upper_quantile = upper_model.predict(x_cali)


In [ ]:
y_cali_lower_quantile

array([1617.79226431, 2143.94746847, 2359.02327303, ..., 2485.71380541,
       2477.99295067, 1848.22388262])

In [ ]:
y_cali_upper_quantile

array([4211.45276931, 4301.2736426 , 4096.0494071 , ..., 4115.98642984,
       4064.48620084, 3826.51719028])

In [ ]:
y_cali_array = np.array(y_cali)

In [ ]:
# lower_diff = np.abs(y_cali_lower_quantile - y_cali_array)
lower_diff = y_cali_lower_quantile - y_cali_array

lower_diff

array([ -990.20773569, -1486.05253153,  -787.97672697, ...,
        -973.28619459,  -697.00704933, -1176.77611738])

In [ ]:
# upper_diff = np.abs(y_cali_upper_quantile - y_cali_array)
upper_diff = y_cali_upper_quantile - y_cali_array

upper_diff

array([1603.45276931,  671.2736426 ,  949.0494071 , ...,  656.98642984,
        889.48620084,  801.51719028])

In [ ]:
stack = np.stack((lower_diff, upper_diff))

In [ ]:
E = np.max(stack, axis=0)

In [ ]:
E

array([1603.45276931,  671.2736426 ,  949.0494071 , ...,  656.98642984,
        889.48620084,  801.51719028])

In [ ]:
len(E)

43233

In [ ]:
(1-alpha)*(1 + 1/(len(E)))

0.9000208174311289

In [ ]:
Q = np.quantile(E, (1-alpha)*(1 + 1/(len(E))))
Q

1398.2381363571164

In [ ]:
E.sort()

In [ ]:
E

array([ 516.50889414,  520.08927636,  539.09827066, ..., 3897.63346023,
       3976.27240263, 4750.08679865])

### Test model

In [ ]:
Q = 0

In [ ]:
test_data = pd.get_dummies(x_test)

y_cali_lower_quantile = lower_model.predict(test_data)
y_cali_upper_quantile = upper_model.predict(test_data)

y_lower_output = y_cali_lower_quantile - Q
y_upper_output = y_cali_upper_quantile + Q


In [ ]:
test_set = pd.DataFrame(y_test)
test_set["pi_lower"] = y_lower_output
test_set["pi_upper"] = y_upper_output
test_set.head()

,DBWT,pi_lower,pi_upper
id,,,
33498,3033,2188.398516,4035.073491
7125,3470,2626.416091,4106.153711
8648,2270,2468.521662,3884.270896
74876,3515,2741.508214,4255.949462
12576,3455,1612.289210,3888.834012


In [ ]:
score_, coverage = score(test_set.DBWT, y_lower_output, y_upper_output, alpha)

In [ ]:
score_

2179.7276402463476

In [ ]:
coverage

0.9009113197946061

## Official submit

In [ ]:
test.head()

,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
id,,,,,,,,,,,,,,,,,,,,,
108082,1,1,23.6,0,11,,8,1231,4,26,...,0,0,0,125,3,1,N,0,M,53
108083,1,1,20.1,0,7,1,4,851,5,32,...,0,2,2,103,4,2,Y,2,F,24
108084,1,1,34.8,30,3,2,12,36,6,24,...,0,2,1,203,1,1,N,0,F,0
108085,1,1,23.4,0,10,2,7,1452,3,32,...,0,1,0,128,1,1,N,0,F,29
108086,1,1,25.4,0,8,,5,803,4,34,...,0,3,0,130,4,1,Y,1,F,18


In [ ]:
official_test_data = pd.get_dummies(test)

In [ ]:
y_cali_lower_quantile = lower_model.predict(official_test_data)
y_cali_upper_quantile = upper_model.predict(official_test_data)

In [ ]:
Q = 3
y_lower_output = y_cali_lower_quantile - Q
y_upper_output = y_cali_upper_quantile + Q
print(Q)

3


In [ ]:
stacked_array = np.stack((y_lower_output, y_upper_output))
result = pd.DataFrame(stacked_array.T, columns=["pi_lower", "pi_upper"])

In [ ]:
result

,pi_lower,pi_upper
0,2328.649248,4245.748478
1,2431.230483,3727.497116
2,2482.946427,3935.585100
3,2583.526009,3880.260623
4,2265.408680,3816.086392
...,...,...
5684,2778.817836,3978.229864
5685,2769.306210,4235.955642
5686,2497.816013,4302.694920
5687,2624.355911,4063.186940


In [ ]:
test_copy = test.copy()

test_copy["pi_lower"]= np.array(result["pi_lower"])
test_copy["pi_upper"]= np.array(result["pi_upper"])

final_output = test_copy[["pi_lower", "pi_upper"]]
final_output.head()

,pi_lower,pi_upper
id,,
108082,2328.649248,4245.748478
108083,2431.230483,3727.497116
108084,2482.946427,3935.585100
108085,2583.526009,3880.260623
108086,2265.408680,3816.086392


In [ ]:
test_copy["pi_lower"]

id
108082    2328.649248
108083    2431.230483
108084    2482.946427
108085    2583.526009
108086    2265.408680
             ...     
113766    2778.817836
113767    2769.306210
113768    2497.816013
113769    2624.355911
113770    2338.070804
Name: pi_lower, Length: 5689, dtype: float64

In [ ]:
final_output.head()

,pi_lower,pi_upper
id,,
108082,2328.649248,4245.748478
108083,2431.230483,3727.497116
108084,2482.946427,3935.585100
108085,2583.526009,3880.260623
108086,2265.408680,3816.086392


In [ ]:
i = 5
print(i)
final_output.to_csv(f"submit_{i}.csv")

5
